In [4]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import re
import pymongo 
import pprint
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'C2ldNUzIEzkdvasD3tlUXWN5W'
        consumer_secret = 'dMa12xOxlWFwo0qcRrJMDbM4ZDSNZslYg8NAcGZeuEFY12RIDD'
        access_token = '922580109185421318-jiCITB5NuplKVmw37sEDbfjvQGF0kzc'
        access_token_secret = 'XlP1Un37MnIDyTFQFkaapcryvBZw10SuassGkQo4TU7X0'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 

 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob((tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count ):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
                parsed_tweet['language'] = tweet.lang
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
               
                # appending parsed tweet to tweets list
            
                tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
            

api=TwitterClient()

connection = pymongo.MongoClient('ds237475.mlab.com',37475)
db = connection['test2']
db.authenticate('aaaaa', 'aaaaa')

#get all tweets that're written in english
cursor =db.us.find({'lang' : 'en'})

ntweets = [document['text'] for document in cursor if api.get_tweet_sentiment(document['text']) =='negative']
cursor.rewind()

ptweets = [document['text'] for document in cursor if api.get_tweet_sentiment(document['text']) =='positive']
cursor.rewind()
neutweets = [document['text'] for document in cursor if api.get_tweet_sentiment(document['text']) =='neutral']
cursor.rewind()
tweets_length=db.us.count({'lang' : 'en'}) +1 

print  " Postive : " , ((len(ptweets)*100)/tweets_length ), " % "
print " Negative  " ,((len(ntweets)*100)/tweets_length ), " % "
print " Neutral :  " ,((len(neutweets)*100)/tweets_length ), " % "
print "  Total  ENGLISH Tweet :  ", (tweets_length )
print (ntweets)

 Postive :  35  % 
 Negative   18  % 
 Neutral :   45  % 
  Total  ENGLISH Tweet :   60
[u'RT  theyearofelan  Donald Trump says we have a mental health problem  not a gun problem  Fortunately  his regime is busy cutting access to ', u'RT  PapiiQuann_  B spent 10 million for this ad to get Donald Trump impeached Yall better Re fucking tweet for the respect                   htt ', u'RT  Channel4News  In February Donald Trump revoked gun checks on people with mental illnesses  Today he blamed a mass shooting on mental he ', u'RT  DartDClark  If Trump s blaming mental health  why did he end this gun check Rule  https   t co bBlbpsiLiR', u'RT  s8n  Comparing Donald Trump to me is really insulting', u'RT  TheLastWord  If Trump s blaming mental health  why did he end this gun check rule   https   t co jflBkEzEMr https   t co ssPVnIZ0cZ', u'RT  JoyAnnReid  Donald Trump signed a bill earlier this year making it easier for the mentally ill to buy guns  https   t co YD8vsxqhxY htt ', u'If Trump 